<a href="https://colab.research.google.com/github/GabrielYamil/TPs_colab/blob/main/TP3/Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%writefile GasMonkeyGarage.cpp
#include <iostream>
#include <atomic>
#include <thread>
#include <queue>
#include <chrono>
#include <vector>
#include <memory>
#include <semaphore>
#include <mutex>

using namespace std;

mutex cout_mutex;  // mutex global solo para sincronizar salida

class TallerMecanico {
private:
    const int N;

    counting_semaphore<6> espacioPlaya{6};         // capacidad playa
    counting_semaphore<6> autosParaInspeccion{0};  // autos listos para inspección
    counting_semaphore<3> espacioFosa{3};          // capacidad en fosa
    counting_semaphore<6> autosParaReparar{0};     // autos listos para reparar
    counting_semaphore<2> espacioServicio{2};      // capacidad de zona de servicio
    counting_semaphore<1> espacioLavado{1};        // un solo lavado a la vez

public:
    TallerMecanico(int n) : N(n) {}

    void richard(int id) {
        espacioPlaya.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Richard lo ingresa a la playa." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        autosParaInspeccion.release();
    }

    void aaron(int id) {
        autosParaInspeccion.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Aaron lo inspecciona." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioFosa.acquire();
        autosParaReparar.release();
    }

    void charles(int id) {
        autosParaReparar.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Charles lo traslada a fosa y repara." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioServicio.acquire();
        espacioFosa.release(); // libera fosa una vez entra a zona de servicio
    }

    void ayudante(int id) {
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Ayudante cambia el aceite." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioServicio.release();

        espacioLavado.acquire();
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "[Auto " << id << "] Se lava el auto." << endl;
        }
        this_thread::sleep_for(chrono::milliseconds(100));
        espacioLavado.release();

        espacioPlaya.release(); // el auto se va
    }

    void ingresarAuto(int id) {
        richard(id);
        aaron(id);
        charles(id);
        ayudante(id);
    }

    void iniciar() {
        vector<thread> threads;
        for (int i = 0; i < N; ++i) {
            threads.emplace_back(&TallerMecanico::ingresarAuto, this, i);
        }
        for (auto& t : threads) {
            t.join();
        }
        {
            lock_guard<mutex> lock(cout_mutex);
            cout << "Todos los autos fueron atendidos y retirados." << endl;
        }
    }
};

int main(int argc, char* argv[]) {
    if (argc != 2) {
        cerr << "Uso: " << argv[0] << " <N_AUTOS>\n";
        return 1;
    }

    int N = stoi(argv[1]);
    TallerMecanico taller(N);
    taller.iniciar();

    return 0;
}

Writing GasMonkeyGarage.cpp


In [9]:
!ls -la

total 92
drwxr-xr-x 1 root root  4096 May 19 03:18 .
drwxr-xr-x 1 root root  4096 May 19 03:03 ..
drwxr-xr-x 4 root root  4096 May 14 13:38 .config
-rw-r--r-- 1 root root  3204 May 19 03:18 GasMonkeyGarage.cpp
drwxr-xr-x 1 root root  4096 May 14 13:38 sample_data
-rwxr-xr-x 1 root root 72520 May 19 03:06 taller


In [6]:
!rm GasMonkeyGarage.cpp

In [10]:
!g++ -std=c++20 -pthread GasMonkeyGarage.cpp -o taller

In [12]:
!./taller 10

[Auto 0] Richard lo ingresa a la playa.
[Auto 3] Richard lo ingresa a la playa.
[Auto 1] Richard lo ingresa a la playa.
[Auto 2] Richard lo ingresa a la playa.
[Auto 4] Richard lo ingresa a la playa.
[Auto 5] Richard lo ingresa a la playa.
[Auto 0] Aaron lo inspecciona.
[Auto 3] Aaron lo inspecciona.
[Auto 4] Aaron lo inspecciona.
[Auto 1] Aaron lo inspecciona.
[Auto 5] Aaron lo inspecciona.
[Auto 2] Aaron lo inspecciona.
[Auto 2] Charles lo traslada a fosa y repara.
[Auto 4] Charles lo traslada a fosa y repara.
[Auto 5] Charles lo traslada a fosa y repara.
[Auto 2] Ayudante cambia el aceite.
[Auto 3] Charles lo traslada a fosa y repara.
[Auto 5] Ayudante cambia el aceite.
[Auto 1] Charles lo traslada a fosa y repara.
[Auto 2] Se lava el auto.
[Auto 4] Ayudante cambia el aceite.
[Auto 3] Ayudante cambia el aceite.
[Auto 0] Charles lo traslada a fosa y repara.
[Auto 5] Se lava el auto.
[Auto 6] Richard lo ingresa a la playa.
[Auto 1] Ayudante cambia el aceite.
[Auto 0] Ayudante cambia e